<a href="https://colab.research.google.com/github/abhishekyadav223/Plant_disease_identification/blob/main/Final_Website.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install flask -q
!pip install tensorflow -q
!pip install gdown -q
!pip install pyngrok -q
!pip install google-generativeai googletrans==3.1.0a0 gTTS -q


import os
import numpy as np
import tensorflow as tf
from PIL import Image
from flask import Flask, request, render_template_string
import gdown
from pyngrok import ngrok
import google.generativeai as genai
from googletrans import Translator
from gtts import gTTS
from IPython.display import Audio, display

In [ ]:
# Download your .h5 model from Google Drive
MODEL_FILE_ID = '1DFAISu8-zx9c9h-iD9wPngKQAoZhV1xx'
MODEL_PATH = 'brinjal_model.h5'

if not os.path.exists(MODEL_PATH):
    url = f'https://drive.google.com/uc?id={MODEL_FILE_ID}'
    gdown.download(url, MODEL_PATH, quiet=False)

model = tf.keras.models.load_model(MODEL_PATH)
print("Model loaded!")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model loaded!


In [ ]:

!pip install flask -q
!pip install tensorflow -q
!pip install gdown -q
!pip install pyngrok -q
!pip install google-generativeai googletrans==3.1.0a0 gTTS -q


import os
import numpy as np
import tensorflow as tf
from PIL import Image
from flask import Flask, request, render_template_string
import gdown
from pyngrok import ngrok
import google.generativeai as genai
from googletrans import Translator
from gtts import gTTS
from IPython.display import Audio, display

In [ ]:
app = Flask(__name__)

UPLOAD_FOLDER = 'uploads'
os.makedirs(UPLOAD_FOLDER, exist_ok=True)

HTML = '''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>🌿 Leaf Disease Detector 🌿</title>
    <style>
        body {
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            margin: 0;
            padding: 0;
            background: linear-gradient(120deg, #56ab2f, #a8e063);
            background-attachment: fixed;
            color: white;
            text-align: center;
        }

        h1 {
            font-size: 2.8rem;
            background: linear-gradient(to right, #ffffff, #d4ff00);
            -webkit-background-clip: text;
            -webkit-text-fill-color: transparent;
            margin-bottom: 20px;
        }

        .container {
            max-width: 900px;
            margin: auto;
            padding: 20px;
        }

        form, .results {
            backdrop-filter: blur(10px);
            background: rgba(0, 0, 0, 0.4);
            border-radius: 20px;
            padding: 25px;
            margin-top: 20px;
            box-shadow: 0 8px 32px rgba(0, 0, 0, 0.37);
        }

        input[type="file"], select, button {
            width: 100%;
            padding: 14px;
            margin: 12px 0;
            border-radius: 12px;
            border: none;
            font-size: 16px;
        }

        select {
            background: white;
            color: black;
        }

        button {
            background-color: #43a047;
            color: white;
            transition: background 0.3s ease;
        }

        button:hover {
            background-color: #2e7d32;
        }

        .columns {
            display: flex;
            flex-wrap: wrap;
            justify-content: space-between;
            gap: 20px;
        }

        .card {
            flex: 1 1 45%;
            padding: 20px;
            border-radius: 15px;
            background: rgba(255, 255, 255, 0.15);
            backdrop-filter: blur(8px);
            box-shadow: 0 4px 20px rgba(0,0,0,0.2);
        }

        audio {
            width: 100%;
            margin-top: 10px;
        }

        img {
            border-radius: 15px;
            max-width: 100%;
            margin-top: 20px;
            box-shadow: 0 4px 15px rgba(0,0,0,0.3);
        }

        .loader {
            margin: 30px auto;
            font-weight: bold;
            font-size: 22px;
            width: fit-content;
            padding: 10px 20px;
            border-radius: 10px;
            background: rgba(255, 255, 255, 0.2);
            animation: pulse 1.2s infinite;
        }

        @keyframes pulse {
            0% { transform: scale(1); }
            50% { transform: scale(1.05); }
            100% { transform: scale(1); }
        }

        @media (max-width: 768px) {
            .columns {
                flex-direction: column;
            }

            .card {
                flex: 1 1 100%;
            }
        }
    </style>
    <script>
        function showLoader() {
            document.getElementById('loader').style.display = 'block';
            document.getElementById('status-text').innerText = 'Uploading Image...';
            setTimeout(() => document.getElementById('status-text').innerText = 'Detecting Disease...', 1000);
            setTimeout(() => document.getElementById('status-text').innerText = 'Generating Remedy via AI...', 2000);
            setTimeout(() => document.getElementById('status-text').innerText = 'Translating Remedy...', 3000);
            setTimeout(() => document.getElementById('status-text').innerText = 'Generating Audio...', 4000);
        }
    </script>
</head>
<body>
    <div class="container">
        <h1>🌿 Leaf Disease Detector</h1>

        <form method="POST" enctype="multipart/form-data" onsubmit="showLoader()">
            <input type="file" name="image" required>
            <select name="language">
                <option value="none" selected>English Only</option>
                <option value="ta">Tamil</option>
                <option value="te">Telugu</option>
                <option value="hi">Hindi</option>
                <option value="ml">Malayalam</option>
                <option value="kn">Kannada</option>
                <option value="gu">Gujarati</option>
                <option value="mr">Marathi</option>
            </select>
            <button type="submit">Detect & Remedy</button>
        </form>

        <div id="loader" class="loader" style="display: none;"><p id="status-text">Processing...</p></div>

        {% if image_path %}
        <img src="{{ image_path }}">
        {% endif %}

        {% if disease %}
        <div class="results">
            <h2>{{ disease }} ({{ disease_translated }})</h2>
            <div class="columns">
                <div class="card">
                    <h3>English Remedy</h3>
                    <audio controls src="/static/remedy_english.mp3"></audio>
                    <p>{{ remedy }}</p>
                </div>
                <div class="card">
                    <h3>Regional Remedy</h3>
                    <audio controls src="/static/remedy_regional.mp3"></audio>
                    <p>{{ remedy_translated }}</p>
                </div>
            </div>
        </div>
        {% endif %}
    </div>
</body>
</html>
'''


In [ ]:
@app.route('/', methods=['GET', 'POST'])
def index():
    disease, remedy, disease_translated, remedy_translated, image_path = None, None, None, None, None
    if request.method == 'POST':
        file = request.files['image']
        language = request.form['language']
        filepath = os.path.join(UPLOAD_FOLDER, file.filename)
        file.save(filepath)
        image_path = '/' + filepath

        img = Image.open(filepath).resize((224, 224))
        img_array = np.expand_dims(np.array(img) / 255.0, axis=0)

        class_idx = np.argmax(model.predict(img_array))
        disease = class_names[class_idx]
        disease_translated = translate_text(disease, language) if language != 'none' else disease
        remedy = get_gemini_remedy(disease, "API KEY")
        remedy_translated = translate_text(remedy, language) if language != 'none' else remedy
        os.makedirs('static', exist_ok=True)
        generate_audio(remedy, 'en', 'static/remedy_english.mp3')
        if language != 'none': generate_audio(remedy_translated, language, 'static/remedy_regional.mp3')

    return render_template_string(HTML, disease=disease, remedy=remedy, disease_translated=disease_translated, remedy_translated=remedy_translated, image_path=image_path)

In [ ]:
ngrok.set_auth_token("2FomXKf1OuvwI6QnKaVMtwKOCCH_7UJAFQwWJ9wiReTrVSSMS")  # Provided ngrok token
public_url = ngrok.connect(5000)
print(f" * ngrok tunnel: {public_url}")

app.run(host='0.0.0.0', port=5000)

 * ngrok tunnel: NgrokTunnel: "https://4359dbc78c49.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [17/Jul/2025 14:36:12] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Jul/2025 14:36:12] "GET /favicon.ico HTTP/1.1" 404 -
